## Mathematische Grundlagen eines PINN
Nachfolgender Programmcode dient dazu die mathematischen Grundlagen eines Physic Informed Neural Networks zu veranschaulichen. 
Insbesondere soll klar werden, wie der Gradient der Kostenfunktion des PINN berechnet werden kann, auch ohne Zuhilfenahme der autograd-Funktion von Pytorch.

In [ ]:
import torch
from torchviz import make_dot

In [ ]:
# Hilfsfunktion zur Berechnung der Sigmoid Funktion
def sigmoid(x):
    return 1/(1 + torch.exp(-x))

In [4]:
# Überprüfung der korrekten Implementierung der Sigmoid Hilfsfunktion
x_0 = torch.tensor(0.0).view(-1, 1)
s_0 = sigmoid(x_0)
s_0

tensor([[0.5000]])

### PINN ohne Aktivierungsfunktion
In untenstehendem Programmcode wird ein PINN mit der Kostenfunktion $C = (\frac{dy}{dt} - 1)^2$ implementiert. Zunächst wird der Ausgabewert des Neuronalen Netzes $y$ berechnet. Dieser wird anschließend mithilfe des Backpropagation Algorithmus nach dem Eingabewert des Neuronalen Netzes $t$ abgeleitet. Die so berechnete Ableitung wird, wie zuvor beschrieben, in die Kostenfunktion integriert. Abschließend wird der Gradient dieser Kostenfunktion, die die Ableitung $\frac{dy}{dt}$ enthält, erneut mittels Backpropagation ermittelt. 

Es ist zu beachten, dass dieses Beispiel insofern vereinfacht ist, dass das Neuronale Netz ohne Aktivierungsfunktion realisiert ist.

In [ ]:
# Belegung der Gewichte, Bias und Eingangswert des NN
a_0 = torch.tensor(2.0, requires_grad=True).view(-1, 1)

W_1 = torch.tensor([1., -1., 2.], requires_grad=True).view(-1, 1)
W_2 = torch.tensor([
    [1., 1., 0.],
    [0., -1., 2.],
    [0., -1., 1]
])
W_3 = torch.tensor([1., 0., 1.], requires_grad=True).view(1, -1)

b_1 = torch.tensor([0., -1., 1.]).view(-1, 1)
b_2 = torch.tensor([-1., -1., 2.]).view(-1, 1)
b_3 = torch.tensor([-1.]).view(-1, 1)

In [ ]:
# Berechnung des Ausgangswertes des NN
a_1 = torch.matmul(W_1, a_0) + b_1
a_2 = torch.matmul(W_2, a_1) + b_2
a_2.requires_grad_(True)
a_3 = torch.matmul(W_3, a_2) + b_3

In [ ]:
# Berechung der Ableitung der Ausgabe des NN nach dem Eingabewert
da3_a0 = torch.autograd.grad(a_3, a_0, create_graph=True)[0]
da3_a0

tensor([[3.]], grad_fn=<TBackward0>)

In [ ]:
# Manuelle Berechnung der Ableitung der Ausgabe nach der Eingabe
c_0 = W_3.t()
c_0

tensor([[1.],
        [0.],
        [1.]], grad_fn=<TBackward0>)

In [168]:
c_1 = torch.matmul(W_2.t(), c_0)
c_1

tensor([[1.],
        [0.],
        [1.]], grad_fn=<MmBackward0>)

In [169]:
c_2 = torch.matmul(W_1.t(), c_1)
c_2

tensor([[3.]], grad_fn=<MmBackward0>)

In [ ]:
# Berechnung der Kostenfunktion mithilfe der zuvor ermittelten Ableitung
C = (da3_a0 - torch.tensor(1.0).view(-1, 1))**2

In [ ]:
# Berechnung der partiellen Ableitung der Kostenfunktion nach den Gewichten W1
dC_dw1 = torch.autograd.grad(C, W_1)
dC_dw1

(tensor([[4.],
         [0.],
         [4.]]),)

In [ ]:
# Manuelle Berechnung der partiellen Ableitung der Kostenfunktion nach den Gewichten W1
dC_dw1_manual = c_1 * 2 * (da3_a0 -1)
dC_dw1_manual

tensor([[4.],
        [0.],
        [4.]], grad_fn=<MulBackward0>)

### PINN mit Aktivierungsfunktion
In untenstehendem Programmcode wird ein PINN mit der Kostenfunktion $C = (\frac{dy}{dt} - 1)^2$ implementiert. Zunächst wird der Ausgabewert des Neuronalen Netzes $y$ berechnet. Dieser wird anschließend mithilfe des Backpropagation Algorithmus nach dem Eingabewert des Neuronalen Netzes $t$ abgeleitet. Die so berechnete Ableitung wird, wie zuvor beschrieben, in die Kostenfunktion integriert. Abschließend wird der Gradient dieser Kostenfunktion, die die Ableitung $\frac{dy}{dt}$ enthält, erneut mittels Backpropagation ermittelt. Im Gegensatz zu obigem PINN wird hierbei die Sigmoid-Funktion als Aktivierungsfunktion eingesetzt. 

Eine Implementierung des Backpropagation-Algorithmus ohne Verwendung der pytorch Funktion autograd, ist für dieses PINN zu komplex. Daher ist der Backpropagation-Algorithmus für ein PINN mit Aktivierungsfunktion im nächsten Abschnitt anhand eines möglichst einfach strukturierten Neuronalen Netz veranschaulicht. 

In [ ]:
# Belegung der Gewichte, Bias und Eingangswert des NN
a_0 = torch.tensor(2.0, requires_grad=True).view(-1, 1)

W_1 = torch.tensor([1., -1., 2.], requires_grad=True).view(-1, 1)
W_2 = torch.tensor([
    [1., 1., 0.],
    [0., -1., 2.],
    [0., -1., 1]
])
W_3 = torch.tensor([1., 0., 1.], requires_grad=True).view(1, -1)

b_1 = torch.tensor([0., -1., 1.]).view(-1, 1)
b_2 = torch.tensor([-1., -1., 2.]).view(-1, 1)
b_3 = torch.tensor([-1.]).view(-1, 1)

In [ ]:
# Berechnung des Ausgabewertes des NN
z_1 = torch.matmul(W_1, a_0) + b_1
a_1 = sigmoid(z_1)
z_2 = torch.matmul(W_2, a_1) + b_2
a_2 = sigmoid(z_2)
a_2.requires_grad_(True)
z_3 = torch.matmul(W_3, a_2) + b_3
a_3 = sigmoid(z_3)

In [ ]:
# Berechung der Ableitung der Ausgabe des NN nach dem Eingabewert
da3_a0 = torch.autograd.grad(a_3, a_0, create_graph=True)[0]
da3_a0

tensor([[0.0042]], grad_fn=<TBackward0>)

In [ ]:
# Manuelle Berechnung der Ableitung dy/dt
a_3 * (1 - a_3)

tensor([[0.2387]], grad_fn=<MulBackward0>)

In [11]:
v_0 = sigmoid(z_3) * (1 - sigmoid(z_3))
c_0 = torch.matmul(W_3.t(), v_0)
c_0

tensor([[0.2387],
        [0.0000],
        [0.2387]], grad_fn=<MmBackward0>)

In [12]:
a_2 * (1 - a_2)

tensor([[0.2497],
        [0.2021],
        [0.0474]], grad_fn=<MulBackward0>)

In [13]:
v_1 = c_0 * sigmoid(z_2) * (1 - sigmoid(z_2))
c_1 = torch.matmul(W_2.t(), v_1)
c_1

tensor([[0.0596],
        [0.0483],
        [0.0113]], grad_fn=<MmBackward0>)

In [14]:
v_2 = c_1 * sigmoid(z_1) * (1 - sigmoid(z_1))
c_2 = torch.matmul(W_1.t(), v_2)
c_2

tensor([[0.0042]], grad_fn=<MmBackward0>)

In [ ]:
# Berechnung der Kostenfunktion und der partiellen Ableitung der Kostenfunktion nach W1
C = (da3_a0 - torch.tensor(1.0).view(-1, 1))**2
dC_dw1 = torch.autograd.grad(C, W_1)
dC_dw1

In [ ]:
#Berechnungsgraph der partiellen Ableitungen nach a_0 und W1
dot = make_dot(C, params={'a_0': a_0, 'W_1': W_1})
dot.render("computation_graph_sigmoid_no_attr", format="png")  # Saves as computation_graph.png


### PINN mit Aktivierungsfunktion aus 3 Neuronen 
Nachfolgender Programmcode dient dazu die partiellen Ableitungen der Kostenfunktion nach den Gewichten $W_1$ und $W_2$ selbst herzuleiten und möglichst ohne die autograd Funktion von Pytorch zu berechnen.

In [ ]:
# Belegung der Variablen des Neuronalen Netzes
a_0 = torch.tensor(2.0, requires_grad=True).view(1, -1)

W_1 = torch.tensor(2.0, requires_grad=True).view(1, -1)
W_2 = torch.tensor(-1.0, requires_grad=True).view(-1, 1)

b_1 = torch.tensor(1).view(1, -1)
b_2 = torch.tensor(2).view(1, -1)


In [ ]:
# Berechnung des Ausgabewertes des Neuronalen Netzes
z_1 = torch.matmul(W_1, a_0) + b_1
a_1 = sigmoid(z_1)
z_2 = torch.matmul(W_2, a_1) + b_2
a_2 = sigmoid(z_2)

In [136]:
# Berechnung der Ableitung der Ausgabe des NN nach der ersten Eingabe (entspricht dy/dt in DGL)
da2_a0 = torch.autograd.grad(a_2, a_0, create_graph=True, retain_graph=True)[0]
da2_a0

tensor([[-0.0026]], grad_fn=<TBackward0>)

In [137]:
# Berechung der Kostenfunktion und partiellen Ableitung nach W2 mithilfe von autograd
y = da2_a0
C = (y- torch.tensor(1.0).view(-1, 1))**2
dC_dw2 = torch.autograd.grad(C, W_2, retain_graph=True)
dC_dw2

(tensor([[-0.0076]]),)

In [138]:
# Berechung der partiellen Ableitung nach W2 vollständig ohne Hilfsmittel, nach der Produktregel
c_1 = a_1 * (1- a_1)
c2 = - (sigmoid(z_2) * (1 - sigmoid(z_2)))
dc_dw2_fully_expanded = (2 * (y-1)) * ((((sigmoid(z_2) * (1- sigmoid(z_2)) * a_1 ) - 2 * a_2 * (sigmoid(z_2) * (1- sigmoid(z_2)) * a_1 )) * W_2) + (a_2 - a_2**2)) * (a_1 - a_1**2) * W_1
dc_dw2_fully_expanded

tensor([[-0.0076]], grad_fn=<MulBackward0>)

In [139]:
# Berechung der partiellen Ableitung nach W2 unter Zuhilfenahme von bereits bei der Bestimmung dy/dt berechneten partiellen Ableitungen
da2_dw2 = torch.autograd.grad(a_2, W_2, create_graph=True, retain_graph=True)[0]
dC_dy = 2 * (y - 1)
c_1 = (a_1 - a_1**2) * W_1
c_2 = (a_2 - a_2**2) * W_2

dC_dW2 = dC_dy * (((da2_dw2 - 2 * a_2 * da2_dw2) * W_2) + (a_2 - a_2**2)) * c_1
dC_dW2

tensor([[-0.0076]], grad_fn=<MulBackward0>)

In [140]:
# Berechung der partiellen Ableitung nach W1 mithilfe von autograd
dC_dw1 = torch.autograd.grad(C, W_1, retain_graph=True)
dC_dw1

(tensor([[-0.0077]]),)

In [144]:
# Berechung der partiellen Ableitung nach W2 unter Zuhilfenahme von bereits bei der Bestimmung dy/dt berechneten partiellen Ableitungen
da2_dw1 = torch.autograd.grad(a_2, W_1, create_graph=True, retain_graph=True)[0]
da1_dw1 = torch.autograd.grad(a_1, W_1, create_graph=True, retain_graph=True)[0]

dC_dy = 2 * (y - 1)
c_1 = (a_1 - a_1**2) * W_1
c_2 = (a_2 - a_2**2) * W_2

dC_dW1 = dC_dy * ( ((da2_dw1 - 2 * a_2 * da2_dw1) * W_2 * c_1) + (((da1_dw1 - 2 * a_1 * da1_dw1) * W_1 * c_2) + (a_1 - a_1**2) * c_2))
dC_dW1

tensor([[-0.0077]], grad_fn=<MulBackward0>)